# Getting Started with LangChain
This notebook shows how to assemble a basic LangChain chain of tasks using the OpenAI API as the backend. This tutorial will require you to have your own OpenAI API key.

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import yaml
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.utilities import WikipediaAPIWrapper

In [2]:
# Loading the API key and organization ID from file (NOT pushed to GitHub)
with open('../keys/openai-keys.yaml') as f:
    keys_yaml = yaml.safe_load(f)

# Setting the OpenAI API key as an environment variable
os.environ['OPENAI_API_KEY'] = keys_yaml['API_KEY']

In [3]:
# Using LangChain to connect to OpenAI
openai_llm = OpenAI()

In [4]:
# Instantiating an object to obtain results from the Wikipedia API
wikipedia_api = WikipediaAPIWrapper(top_k_results = 1)

## Prompt Templates
In order to create a chained set of tasks, we need to instantiate the individual tasks with **prompt templates**. Prompt templates combine an input alongside some preset feature engineering to produce a precise result (also known as a Completion).

In [12]:
# Creating a prompt template for checking to see if the inputted individual is a historical figure
is_historical_figure_template = PromptTemplate(
    input_variables = ['entity_name'],
    template = 'Is the following entity a person, and if yes, would you consider this person to be a historical figure: {entity_name}. Please give me back just a yes or no answer.'
)

# Creating a prompt template for generating a research paper outline the three most important events in the historical figure's life
research_paper_outline_template = PromptTemplate(
    input_variables = ['entity_name', 'wikipedia_entry'],
    template = 'The following is a Wikipedia entry about {entity_name}. Please provide for me an outline of a basic research paper with an introduction, the three most important events of this person\'s life, and a conclusion. Here is the Wikipedia information:\n\n {wikipedia_entry}'
)

# Creating a prompt template for generating a research paper based on the outline generated from the previous prompt template
research_paper_template = PromptTemplate(
    input_variables = ['entity_name', 'research_paper_outline'],
    template = 'You are a high schooler who has just been assigned research paper about a historical figure. The historical figure is {entity_name}, and the following is an outline to follow:\n {research_paper_outline}. Please write a research paper approximately three pages in length.'
)

# Creating a prompt template for re-writing the research paper as Jar Jar Binks
jar_jar_template = PromptTemplate(
    input_variables = ['research_paper'],
    template = 'Please rewrite the following research paper in the tone of Jar Jar Binks from Star Wars:\n\n {research_paper}'
)

In [13]:
# Instantiating the LangChains objects for all the prompt templates defined above
is_historical_figure_chain = LLMChain(llm = openai_llm, prompt = is_historical_figure_template, verbose = True, output_key = 'is_historical_figure')
research_paper_outline_chain = LLMChain(llm = openai_llm, prompt = research_paper_outline_template, verbose = True, output_key = 'research_paper_outline')
research_paper_chain = LLMChain(llm = openai_llm, prompt = research_paper_template, verbose = True, output_key = 'research_paper')
jar_jar_chain = LLMChain(llm = openai_llm, prompt = jar_jar_template, verbose = True, output_key = 'jar_jar_paper')

## Instantiating the LangChain

In [14]:
# Creating the LangChain chain
research_paper_langchain = SequentialChain(chains = [research_paper_outline_chain, research_paper_chain, jar_jar_chain],
                                           input_variables = ['entity_name', 'wikipedia_entry'],
                                           output_variables = ['research_paper_outline', 'research_paper', 'jar_jar_paper'],
                                           verbose = True)

## Using the LangChain

In [15]:
# Setting an entity for test purposes
entity_name = 'Steve Jobs'

In [16]:
# Getting the wikipedia information associated to the historical figure
wikipedia_entry = wikipedia_api.run(f'{entity_name} (person)')

In [17]:
# Passing the wikipedia entry and historical figure name into the LangChain
response = research_paper_langchain({'entity_name': entity_name, 'wikipedia_entry': wikipedia_entry})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
The following is a Wikipedia entry about Steve Jobs. Please provide for me an outline of a basic research paper with an introduction, the three most important events of this person's life, and a conclusion. Here is the Wikipedia information:

 Page: Steve Jobs
Summary: Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American business magnate, inventor, and investor. He was the co-founder, chairman, and CEO of Apple; the chairman and majority shareholder of Pixar; a member of The Walt Disney Company's board of directors following its acquisition of Pixar; and the founder, chairman, and CEO of NeXT. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.
Jobs was born in San Francisco to a Syrian father and German-American mother. He was adopted shortly after his birth. Job

In [18]:
response

{'entity_name': 'Steve Jobs',
 'wikipedia_entry': 'Page: Steve Jobs\nSummary: Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American business magnate, inventor, and investor. He was the co-founder, chairman, and CEO of Apple; the chairman and majority shareholder of Pixar; a member of The Walt Disney Company\'s board of directors following its acquisition of Pixar; and the founder, chairman, and CEO of NeXT. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\nJobs was born in San Francisco to a Syrian father and German-American mother. He was adopted shortly after his birth. Jobs attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to sell Wozniak\'s Apple I personal computer. Together the duo gained fame and wealth a year